[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_11_clustering-published.ipynb) (Este botón no anda, tenemos que ver dónde colgaremos las cosas)

# Práctica de herramientas pre-curso

En este notebook refrescaremos (o exploraremos por primera vez) herramientas para el curso `Inferencia bayesiana causal` del [Departamento de Computación](https://www.dc.uba.ar/) de la [Facultad de Ciencias Exactas y Naturales](https://exactas.uba.ar/) de la [Universidad de Buenos Aires](https://uba.ar/).



## Instalando dependencias

En esta materia usaremos el lenguaje Python y algunas bibliotecas que mencionamos en este notebook.

In [ ]:
!pip install scipy
!pip install pandas
!pip install plotnine
!pip install seaborn
!pip install scikit-learn

## Obteniendo muestras

Para realizar un muestreo de distintas distribuciones utilizaremos el paquete `scipy`.

> SciPy es una colección de algoritmos matemáticos y funciones de conveniencia construidas sobre NumPy. Añade un poder significativo a Python al proporcionar al usuario comandos y clases de alto nivel para manipular y visualizar datos. 
Fuente: [Página oficial scipy](https://docs.scipy.org/doc/scipy/tutorial/index.html)

En particular usaremos el módulo `stats` que puede importarse de la siguiente manera

In [ ]:
from scipy import stats


De esta manera podemos generarnos una variable aleatoria con una distribución que elijamos para utilizar. 
Empecemos con una [uniforme](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html#scipy.stats.uniform) (recordá que siempre podés ver la documentación ;-)).

$$X \sim Unif(a,b)$$

Que quiere decir que la variable X es uniforme en el intervalo $[a, b]$.


In [ ]:
a = 0
b = 1
X = stats.uniform(a, b-a)

De esta manera podemos usar `X` para obtener diferentes cosas:

 - `rvs(loc=0, scale=1, size=1, random_state=None)` obtener muestras de la distribución
 - `pdf(x, loc=0, scale=1)` obtiene el valor de la densidad
 - `logpdf(x, loc=0, scale=1)` idem anterior pero en escala $log$
 - `cdf(x, loc=0, scale=1)` función de distribución acumulada

**Ejercicio**: 

Obtener, mediante una muestra de 1000 puntos equidistantes en el intervalo $[1.5, 3.5]$, el valor de la densidad puntual y la acumulada de una uniforme(2, 3).
Además obtener mil muestras de dicha variable aleatoria.

Consejo: para este ejericio explorar la función linspace de numpy

In [ ]:
import numpy as np

X = ...

xs = np.linspace(...)
dp = ...
dacum = ...
muestras = ..

# posible idea de resolucion
for x in xs:
    ... COMPLETAR ...

Veamos de graficar los valores obtenidos.

Para esto será necesario incluir entre nuestras herramientas algún paquete graficador. Existen varios: [matplotlib](https://matplotlib.org/), [seaborn](https://seaborn.pydata.org/), [plotnine](https://plotnine.org/tutorials/). 
En este caso exploraremos el último, por su potencia y por su gramática de construcción. Originariamente fue desarrollado en el marco del lenguaje R.

La idea de un gráfico en `plotnine` consiste en atar datos al gráfico y luego aplicarle capas que usan esos datos. En las sucesivas capas es posible ir modificando cosas, incluso cambiar los datos para el mismo gráfico. Por lo general toma como entrada un `DataFrame` de `pandas` por lo que aprovechamos para introducir esta estructura también.

Realicemos 3 gráficos.

In [ ]:
import pandas as pd
from plotnine import ggplot, aes, ggtitle, geom_point, facet_grid, geom_histogram, ylab, theme_xkcd, geom_abline, xlim, ylim, geom_density_2d # completar con las funciones que uses

In [ ]:
g1 = (
    ggplot(pd.DataFrame({'x': xs, 'densidad': dp}), aes(x='x', y='densidad')) +
    geom_point() +
    ggtitle('Funcion de densidad de probabilidad') +
    theme_xkcd()
)
print(g1)

In [ ]:
g2 = (
    ggplot(pd.DataFrame({'x': xs, 'acumulada': dacum}), aes(x='x', y='acumulada')) +
    geom_point() +
    ggtitle('Funcion de acumulada') +
    theme_xkcd()
)
print(g2)

Para graficar las muestras podemos hacer un histograma, que divide el espacio en segmentos y cuenta cuántos puntos hay en cada uno de ellos.

In [ ]:
g3 = (
    ggplot(pd.DataFrame({'Valores': muestras}), aes(x='Valores')) +
    geom_histogram(binwidth=0.05) +
    ggtitle('Muestras') +
    theme_xkcd() +
    ylab("Cantidad")
)
print(g3)

**Ejercicio**: Explorar el paquete `scipy` y repetir el ejercicio anterior pero para una variable aleatoria con [distribución normal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm) con todas las combinaciones de parámetros:

  - media: -1, 0, 1
  - desvío estandar: 0.5, 1, 1.5

**Ejercicio**: Simular y generar datos que tengan 2 dimensiones independientes y que a su vez vengan de Normales:

   - Una con media 5 y desvío 0.5
   - La segunda con media -1 y desvío 3

Generar mil datos y graficarlos cuidando de mantener la relación 1 a 1 entre los ejes (es decir, tanto el eje X como el Y deben mostrar el mismo intervalo).

# Lectura de datos

En esta sección vamos a leer datos de un archivo, modificarlos, hacer cálculos y graficarlos. Este conjunto de datos presenta la altura de una persona, su género, la contextura física de la madre y la altura de la misma.

In [ ]:
# df = pd.read_csv('https://raw.githubusercontent.com/emmanueliarussi/DataScienceCapstone/master/3_MidtermProjects/ProjectWC/data/attacks.csv', encoding = "ISO-8859-1")
df = pd.read_csv('data_00_alturas.csv')

df.head()

En una nueva columna llamada `diferencia` calcular la diferencia entre la altura de la persona y la de la madre.

In [ ]:
df['diferencia'] = df["altura"] - df["altura_madre"]
df.head()

Grafiquemos la diferencia

In [ ]:
g1 = (
    ggplot(df, aes(x='diferencia')) +
    geom_histogram(binwidth=4) +
    ggtitle('Diferencia de altura a la madre') +
    theme_xkcd()
)
print(g1)

Grafiquemos la diferencia pero separada por género.

In [ ]:
g1 = (
    ggplot(df, aes(x='diferencia', fill="genero")) +
    geom_histogram(binwidth=4, alpha=0.6, color="black", position = 'identity') +
    ggtitle('Diferencia de altura a la madre') +
    theme_xkcd()
)
print(g1)

**Ejercicio**: Encontrar los mejores $\beta_0$ y $\beta_1$ para que en este conjunto de datos la formula:

$$ \text{altura} = \beta_0 + \text{altura\_madre} \times \beta_1 $$

Para ello explorar (varios o muchos) valores. Elegir un intervalo de exploración razonable, sino se te ocurre cual podés probar entre 10 y 15 para $\beta_0$ y entre -0.5 y 1.5 para $\beta_1$.

1. ¿Qué criterio usaste para elegir el mejor?

1. Realizar un gráfico siguiendo los siguientes lineamientos:

    - en el eje x esté la altura de la madre
    - en el eje y este la altura
    - cada persona sea un punto (determinado por los ejes)
    - calcular los valores de la recta definida según la ecuación anterior y graficarla sobre los puntos